# import library

In [83]:
# import all library
import librosa, IPython, datetime, time, os, sys, copy, dill, mir_eval, glob
#import pickle
import _pickle as pickle
import numpy as np
#import pandas as pd
#from scipy.spatial.distance import euclidean, pdist, squareform
#import IPython.display as ipd
#from datetime import datetime
from time import gmtime, strftime
#from imageio import imread as imread
#from imageio import imsave as imsave
#import librosa.display
import tensorflow as tf
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
#from midiutil.MidiFile import MIDIFile
#import ray
from ops import *
#import pydub
#from ADTLib import ADT
import matplotlib.pyplot as plt
%matplotlib inline

# show version info
print ("[info] Current Time:     " + datetime.datetime.now().strftime('%Y/%m/%d  %H:%M:%S'))
print ("[info] Python Version:   " + sys.version.split('\n')[0].split(' ')[0])
print ("[info] Working Dir:      " + os.getcwd()+'/')

# enable gpu usage constraint here
fixed_gpu_usage = 0

# if gpu usage is constraint, limit certain gpu for use
if (fixed_gpu_usage == 1):
    # set available GPU
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"        # list GPU sequence by PCI bus GPU ID
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"         # 0 for GPU 0(1080Ti), 1 for GPU 1(1080Ti), 2 for GPU 2(2080Ti)
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"              # 0 for GPU 0(1080Ti), 1 for GPU 1(1080Ti), 2 for GPU 2(2080Ti)

    # check available GPU
    from tensorflow.python.client import device_lib
    for x in range(1, len(device_lib.list_local_devices())):
        print ("[info] GPU " + device_lib.list_local_devices()[x].physical_device_desc)

[info] Current Time:     2019/03/03  13:11:28
[info] Python Version:   3.6.5
[info] Working Dir:      /host/home/python/musegan_npz/


# Read test s100 index code

In [84]:
# read test s100 data
with open('./large_dataset_data/cqt_obj_pkl/song_index_s100_test_list.pkl', 'rb') as pkl_file:        
    song_index_s100_test_list_reload = pickle.load(pkl_file)
    
print ('[info] test s100 pkg is loaded.')

s100_song_index_list_test =     song_index_s100_test_list_reload[0]
s100_song_bar_num_list_test =   song_index_s100_test_list_reload[1]
s100_index_code_list_test =     song_index_s100_test_list_reload[2]

print (len(s100_song_index_list_test))
print (s100_song_index_list_test[:5])
print (len(s100_song_bar_num_list_test))
print (np.sum(s100_song_bar_num_list_test))
print (len(s100_index_code_list_test))
print(s100_index_code_list_test[:5])

[info] test s100 pkg is loaded.
100
['00244', '00353', '00367', '00395', '00680']
100
11338
11338
['00244000sp0', '00244001sp0', '00244002sp0', '00244003sp0', '00244004sp0']


# check sampled file index

In [625]:
print('s100 index: {:2d}'.format(s100_song_index_list_test.index('00244')))
print('s100 index: {:2d}'.format(s100_song_index_list_test.index('01247')))
print('s100 index: {:2d}'.format(s100_song_index_list_test.index('03216')))
print('s100 index: {:2d}'.format(s100_song_index_list_test.index('05184')))
print('s100 index: {:2d}'.format(s100_song_index_list_test.index('07942')))
pick_song_index_list = [0, 8, 23, 46, 76, 58, 81, 63, 95, 14]


pick_song_index = pick_song_index_list[9]
model_version = 'v033'
add_note_ver = 'p06n'

s100 index:  0
s100 index:  8
s100 index: 23
s100 index: 46
s100 index: 76


# reload s100 test result

In [626]:
# reload test result
test_s100_result_fname = './large_dataset_data/model_test_result/{}/test_s100_{}_result_pkg.pkl'.format(model_version, 
                                                                                                        add_note_ver)

chkpt_ver = test_s100_result_fname.split('/')[-1].split('_')[-3]

with open(test_s100_result_fname, 'rb') as pkl_file:
    test_s100_result_pkg = pickle.load(pkl_file)

# reload s100 train test data
bar_cqt_data_train_list =        test_s100_result_pkg[0][0]
bar_note_num_train_list =        test_s100_result_pkg[0][1]
bar_note_num_pred_train_list =   test_s100_result_pkg[0][2]
bar_arrange_train_list =         test_s100_result_pkg[0][3]
darr_output_train_list =         test_s100_result_pkg[0][4]

# reload s100 test test data
bar_cqt_data_test_list =         test_s100_result_pkg[1][0]
bar_note_num_test_list =         test_s100_result_pkg[1][1]
bar_note_num_pred_test_list =    test_s100_result_pkg[1][2]
bar_arrange_test_list =          test_s100_result_pkg[1][3]
darr_output_test_list =          test_s100_result_pkg[1][4]

print ('s100 test data is reloaded.')

s100 test data is reloaded.


## show darr output test data format

In [627]:
bar_cqt_data_test_ary = np.concatenate(bar_cqt_data_test_list, axis=0)
bar_arrange_test_ary = np.concatenate(bar_arrange_test_list, axis=0)
darr_output_test_ary = np.concatenate(darr_output_test_list, axis=0)
darr_output_test_ary_bin = darr_output_test_ary.copy()

note_thv = 0.5
darr_output_test_ary_bin[darr_output_test_ary_bin>=note_thv] = 1.0
darr_output_test_ary_bin[darr_output_test_ary_bin<note_thv] = 0.0

print('s100 test CQT:          {}'.format(bar_cqt_data_test_ary.shape))
print('s100 test Drum GT:      {}'.format(bar_arrange_test_ary.shape))
print('s100 test Drum Model:   {}'.format(darr_output_test_ary.shape))

s100 test CQT:          (11328, 84, 96)
s100 test Drum GT:      (11328, 46, 16)
s100 test Drum Model:   (11328, 46, 16)


# show s100 test data distribution

In [628]:
print ('[info] GT data')
print('Bar value mean: {:.6f}'.format(np.mean(bar_arrange_test_ary)))
print('Bar value std: {:.6f}'.format(np.std(bar_arrange_test_ary)))

print ('\n[info] Model out bin')
print('Bar value mean: {:.6f}'.format(np.mean(darr_output_test_ary_bin)))
print('Bar value std: {:.6f}'.format(np.std(darr_output_test_ary_bin)))

[info] GT data
Bar value mean: 0.026878
Bar value std: 0.161727

[info] Model out bin
Bar value mean: 0.022580
Bar value std: 0.148559


# Define function to get complete single song index (start, end)

In [629]:
s100_song_index_list_test =     song_index_s100_test_list_reload[0]
s100_song_bar_num_list_test =   song_index_s100_test_list_reload[1]
s100_index_code_list_test =     song_index_s100_test_list_reload[2]

def get_test_song_abs_idx(pick_song_index):

    song_index_in_s100 = s100_song_index_list_test[pick_song_index]

    song_index_all_bars = [x for x in s100_index_code_list_test if x[0:5]==song_index_in_s100]

    bar_idx_start = s100_index_code_list_test.index(song_index_all_bars[0])
    bar_idx_end = s100_index_code_list_test.index(song_index_all_bars[-1])
    
    return ([song_index_in_s100, bar_idx_start, bar_idx_end+1])

print ('Function define done')

Function define done


# get song 0 drum arrange data

In [630]:
model_darr_list = []

# get complete single song index data
pick_song_index = pick_song_index
abs_song_idx, abs_idx_start, abs_idx_end = get_test_song_abs_idx(pick_song_index)

print ('[info] Song index: {}'.format(abs_song_idx))
print ('[info] Song bars: {}'.format(abs_idx_end - abs_idx_start))
print ('[info] Abs start index: {}'.format(abs_idx_start))
print ('[info] Abs end index: {}'.format(abs_idx_end))
print('')

# plot complete single song drum arrangement
bar_idx_start = abs_idx_start
bar_idx_end = abs_idx_end

for bar_idx in range(bar_idx_start, bar_idx_end):
    
    #plot_spectrogram =      bar_cqt_data_test_ary[bar_idx,:,:]
    #plot_dataset_gt_darr =  bar_arrange_test_ary[bar_idx,:,:]
    plot_model_out_darr =   darr_output_test_ary_bin[bar_idx,:,:]
    
    model_darr_list.append(plot_model_out_darr)
    
    
print ('model darr list len: {}'.format(len(model_darr_list)))
print ('darr bar format: {}'.format(model_darr_list[0].shape))

[info] Song index: 02200
[info] Song bars: 154
[info] Abs start index: 1670
[info] Abs end index: 1824

model darr list len: 154
darr bar format: (46, 16)


# define reconstruct odrm data function (96, 128)

In [631]:
# keep 99 % of all instrument count (total 46 insts)
selected_inst_list_46 = [27, 28, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, \
                         51, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 73, \
                         74, 75, 76, 77, 80, 81, 82, 83, 85, 87]
print ('keeped Insts: {}'.format(len(selected_inst_list_46)))

def get_odrum_shape(drum_ary_in):    
    odrum_data = np.zeros([96,128])
    for x in range(0, drum_ary_in.shape[0]):
        for y in range(0, drum_ary_in.shape[1]):            
            pix_value = drum_ary_in[x,y]
            if pix_value>0.5:
                odrum_data[y*6, selected_inst_list_46[x]] = 100
            
    return (odrum_data)

print ('get_odrum_shape is defined.')

keeped Insts: 46
get_odrum_shape is defined.


# convert drum data into original shape (96, 128)

In [632]:
model_darr_odrm_list = [get_odrum_shape(x) for x in model_darr_list]
model_darr_odrm_ary = np.concatenate(model_darr_odrm_list, axis=0)
print(model_darr_odrm_ary.shape)

(14784, 128)


# find original npz file

In [633]:
#Read all npz files
#npz_flist = np.sort(glob.glob('./lpd_5_cleansed/**/*.npz', recursive=True)).tolist()
#print ('Total files: {}'.format(len(npz_flist)))

# show file format
#for x in npz_flist[:3]:
#    print (x)

#with open('./ld_npz_file_name_list.pkl', 'wb') as pkl_file:
#    pickle.dump(npz_flist, pkl_file)

with open('./ld_npz_file_name_list.pkl', 'rb') as pkl_file:
    npz_flist = pickle.load(pkl_file)
    
# Reload good npz / genre index list
file_name = './ld_good_npz_files_genre_info.pkl'
with open(file_name, 'rb') as pkl_file:
    good_npz_genre_list = pickle.load(pkl_file)
    
file_unique_genre = good_npz_genre_list[0]
file_idx_with_genre_list = good_npz_genre_list[1]
file_genre_name_list = good_npz_genre_list[2]
file_genre_index_list = good_npz_genre_list[3]

#print ('Good npz files: {}'.format(len(file_idx_with_genre_list)))
print(file_idx_with_genre_list[:12])
print(file_genre_name_list[:12])
print(file_genre_index_list[:12])
print('')

print ('Genre num: {}'.format(len(file_unique_genre)))
#for k, x in enumerate(file_unique_genre):
#    print('ID: {}, Name: {}'.format(k, x))

[0, 1, 2, 5, 6, 7, 9, 10, 12, 15, 18, 22]
['Pop', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'Country', 'unknown', 'unknown', 'unknown', 'unknown', 'Rock']
[8, 15, 15, 15, 15, 15, 1, 15, 15, 15, 15, 13]

Genre num: 16


# Get original NPZ file name

In [634]:
onpz_fname = npz_flist[file_idx_with_genre_list[int(abs_song_idx)]]
print (onpz_fname)

./lpd_5_cleansed/F/O/M/TRFOMJV128F424EBAC/fa08cc5d36c7ed6c73a2d156c04f582b.npz


# ensure DIR

In [635]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# write midi back on disk

In [636]:
mtrack_data = Multitrack(onpz_fname)

# prepair writen data
new_drum_notes = model_darr_odrm_ary
    
# write drum notes in multitrack object
mtrack_data.append_track(track=None, 
                         pianoroll=new_drum_notes, 
                         program=5, 
                         is_drum=True,
                         name='Drums_{}'.format(chkpt_ver))

# transfer data into pretty midi format
pmidi_data = mtrack_data.to_pretty_midi(constant_tempo=None)

# print instruments
print ('[info] Show {} Insts.'.format(len(pmidi_data.instruments)))
for x in pmidi_data.instruments:
    print (x)
print('')

# make all notes in Drums2 velocity=99
for instrument in pmidi_data.instruments:
    if instrument.program==5:
        for note in instrument.notes:
            note.velocity = 120


# set midi file name to write
midi_file_name = './large_dataset_data/write_midi_back/{}/song_10k_id_{}_{}.mid'.format(model_version, 
                                                                                        abs_song_idx, 
                                                                                        chkpt_ver)

# create folder if not exist
ensure_dir(midi_file_name)

print('[info] Saving file: \"{}\" ...'.format(midi_file_name))
# write midi file
pmidi_data.write(midi_file_name)

print ('[info] \"{}\" is saved.'.format(midi_file_name))

[info] Show 6 Insts.
Instrument(program=0, is_drum=True, name="Drums")
Instrument(program=0, is_drum=False, name="Piano")
Instrument(program=24, is_drum=False, name="Guitar")
Instrument(program=32, is_drum=False, name="Bass")
Instrument(program=48, is_drum=False, name="Strings")
Instrument(program=5, is_drum=True, name="Drums_p06n")

[info] Saving file: "./large_dataset_data/write_midi_back/v033/song_10k_id_02200_p06n.mid" ...
[info] "./large_dataset_data/write_midi_back/v033/song_10k_id_02200_p06n.mid" is saved.


# Plot complete single song (index=0)

In [ ]:
# get complete single song index data
pick_song_index = 
abs_song_idx, abs_idx_start, abs_idx_end = get_test_song_abs_idx(pick_song_index)

print ('Song index: {}'.format(abs_song_idx))
print ('Song bars: {}'.format(abs_idx_end - abs_idx_start))
print ('Abs start index: {}'.format(abs_idx_start))
print ('Abs end index: {}'.format(abs_idx_end))


# plot complete single song drum arrangement
bar_idx_start = abs_idx_start
bar_idx_end = abs_idx_end
#bar_idx_end = abs_idx_start + 32

for bar_idx in range(bar_idx_start, bar_idx_end):
    
    plot_spectrogram =      bar_cqt_data_test_ary[bar_idx,:,:]
    plot_dataset_gt_darr =  bar_arrange_test_ary[bar_idx,:,:]
    plot_model_out_darr =   darr_output_test_ary_bin[bar_idx,:,:]
    
    print ('song bar idx: [ {} / {} ]'.format(bar_idx - bar_idx_start + 1, bar_idx_end - bar_idx_start))

    plot_data0 = plot_spectrogram
    plt.figure(figsize=(16,3))
    plt.imshow(plot_data0, interpolation='nearest', origin='lower', aspect='auto', cmap='hot')
    plt.grid()
    plt.show()

    #plot_data1 = np.where(plot_dataset_gt_darr>=0.5,
    #                      np.ones_like(plot_dataset_gt_darr),
    #                      np.zeros_like(plot_dataset_gt_darr))                      
    plot_data1 = plot_dataset_gt_darr
                          
    plt.figure(figsize=(16,3))
    plt.imshow(plot_data1, interpolation='nearest', origin='lower', aspect='auto', cmap='hot')
    plt.xticks(np.arange(0, plot_data1.shape[1], 1), rotation=0)
    plt.yticks(np.arange(0, plot_data1.shape[0], 2))
    #plt.ylim(20,80)
    plt.grid()
    plt.show()

    
    plot_data2 = plot_model_out_darr
    
    plt.figure(figsize=(16,3))
    plt.imshow(plot_data2, interpolation='nearest', origin='lower', aspect='auto', cmap='hot')
    plt.xticks(np.arange(0, plot_data2.shape[1], 1), rotation=0)
    plt.yticks(np.arange(0, plot_data2.shape[0], 2))
    #plt.ylim(20,80)
    plt.grid()
    plt.show()